In [3]:
import requests
import csv
from datetime import datetime, timedelta

url = "https://pasardana.id/api/StockSearchResult/GetAll"

start_date = datetime(2025, 3, 10)  
end_date = datetime(2025, 3, 12)    

headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
    "Host": "pasardana.id",
    "Pragma": "no-cache",
    "Referer": "https://pasardana.id/stock/search",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

# Simpan semua data yang diambil
all_data = []

# Looping untuk setiap tanggal dalam rentang
for i in range((end_date - start_date).days + 1):
    tanggal_dicari = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")

    # Parameter API
    params = {
        "search": "MLIA",  # Kode saham yang diinginkan
        "page": 1,
        "per_page": 100,
        "sort_by": "Code",
        "sort_direction": "asc",
        "date": tanggal_dicari,  # Filter tanggal
    }

    # Kirim permintaan ke API
    response = requests.get(url, params=params, headers=headers)

    # Cek respons
    if response.status_code == 200:
        data = response.json()  # Data berbentuk list

        filtered_data = [
            {
                "Frequency": stock.get("Frequency", 0)
            }
            for stock in data
            if stock["Code"] == "MLIA" and stock["LastDate"][:10] == tanggal_dicari
        ]

        # Gabungkan hasilnya
        all_data.extend(filtered_data)
    else:
        print(f"Error {response.status_code} pada tanggal {tanggal_dicari}, Message: {response.text}")

# Nama file CSV untuk menyimpan data
csv_filename = "data_saham_MLIA.csv"

# Simpan ke file CSV
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(
        file,
        fieldnames=["Frequency"]
    )
    writer.writeheader()  # Tulis header CSV
    writer.writerows(all_data)  # Tulis data saham

print(f"✅ Data berhasil disimpan ke {csv_filename}")

✅ Data berhasil disimpan ke data_saham_MLIA.csv


In [ ]:
import requests
import pandas as pd
import os

# URL API
url = "https://pasardana.id/api/StockSearchResult/GetAll"

# Header untuk request API
headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
    "Host": "pasardana.id",
    "Pragma": "no-cache",
    "Referer": "https://pasardana.id/stock/search",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

# Daftar saham yang ingin diperbarui
saham_list = ["BBCA", "TLKM", "BBRI", "BMRI"]  # Bisa ditambah sesuai kebutuhan

# Direktori penyimpanan data CSV
csv_dir = "data_saham"

# Pastikan folder penyimpanan CSV ada
os.makedirs(csv_dir, exist_ok=True)

# Fungsi untuk mengambil data saham dari API
def get_stock_data(stock_code):
    params = {
        "search": stock_code,
        "page": 1,
        "per_page": 100,
        "sort_by": "Code",
        "sort_direction": "asc",
    }
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json().get("Data", [])
        return [
            {
                "Date": stock["LastDate"],
                "Open": stock["AdjustedOpenPrice"],
                "High": stock["AdjustedHighPrice"],
                "Low": stock["AdjustedLowPrice"],
                "Close": stock["AdjustedClosingPrice"],
                "Volume": stock["Volume"],
                "Frequency": stock["Frequency"],
            }
            for stock in data
            if "LastDate" in stock
        ]
    else:
        print(f"Error fetching {stock_code}: {response.status_code}")
        return []

# Fungsi untuk memperbarui data CSV saham
def update_csv(stock_code, new_data):
    csv_file = os.path.join(csv_dir, f"data_saham_{stock_code}.csv")

    # Jika file tidak ada, buat file baru dengan data baru
    if not os.path.exists(csv_file):
        df = pd.DataFrame(new_data)
        df.to_csv(csv_file, index=False)
        print(f"✅ File baru dibuat: {csv_file}")
        return

    # Jika file sudah ada, load data lama
    df_old = pd.read_csv(csv_file, parse_dates=["Date"])
    df_new = pd.DataFrame(new_data)

    # Gabungkan data lama dan baru, lalu hapus duplikasi berdasarkan "Date"
    df_combined = pd.concat([df_old, df_new]).drop_duplicates(subset=["Date"]).sort_values(by="Date")

    # Simpan kembali ke file CSV
    df_combined.to_csv(csv_file, index=False)
    print(f"🔄 Data diperbarui: {csv_file}")

# **Proses update untuk setiap saham**
for stock in saham_list:
    print(f"📊 Mengupdate {stock}...")
    stock_data = get_stock_data(stock)
    if stock_data:
        update_csv(stock, stock_data)
    else:
        print(f"⚠️ Tidak ada data baru untuk {stock}.")

print("✅ Update selesai!")


In [ ]:
import pandas as pd
import glob
import os
import plotly.graph_objects as go

# Path ke folder tempat menyimpan file saham
folder_path = "data saham/"

# Cari semua file Ringkasan Saham di folder
file_list = glob.glob(os.path.join(folder_path, "Ringkasan Saham-*.xlsx"))

if not file_list:
    print("❌ Tidak ditemukan file Ringkasan Saham di folder 'data saham'")
else:
    print(f"📂 Ditemukan {len(file_list)} file saham")

    # Buat list untuk menyimpan semua data
    all_data = []

    # Loop melalui setiap file dan baca datanya
    for file in file_list:
        df = pd.read_excel(file)

        # Ekstrak tanggal dari nama file (format: Ringkasan Saham-YYYYMMDD.xlsx)
        filename = os.path.basename(file)
        date_str = filename.split("-")[-1].split(".")[0]  # Ambil bagian tanggal dari nama file
        df["Tanggal"] = pd.to_datetime(date_str, format="%Y%m%d")  # Konversi ke datetime

        all_data.append(df)

    # Gabungkan semua data menjadi satu DataFrame
    df_all = pd.concat(all_data, ignore_index=True)

    # Pastikan kolom yang dibutuhkan ada
    required_columns = {"Kode Saham", "Open Price", "Tertinggi", "Terendah", "Penutupan", "Volume", "Frekuensi"}
    if not required_columns.issubset(df_all.columns):
        print("⚠️ Struktur kolom tidak sesuai. Pastikan file memiliki kolom:", required_columns)
    else:
        # Pilih kode saham tertentu (misalnya, "SSIA")
        selected_stock = "SSIA"
        df_selected = df_all[df_all["Kode Saham"] == selected_stock].copy()

        if df_selected.empty:
            print(f"⚠️ Tidak ada data untuk kode saham {selected_stock}")
        else:
            # Urutkan data berdasarkan tanggal
            df_selected = df_selected.sort_values(by="Tanggal")

            # **ISI TANGGAL KOSONG** agar grafik tetap kontinu
            full_date_range = pd.date_range(start=df_selected["Tanggal"].min(), end=df_selected["Tanggal"].max())
            df_selected = df_selected.set_index("Tanggal").reindex(full_date_range).reset_index()
            df_selected.rename(columns={"index": "Tanggal"}, inplace=True)

            # **GUNAKAN DATA TERAKHIR UNTUK MENGISI KEKOSONGAN** (forward-fill)
            df_selected.fillna(method="ffill", inplace=True)

            # Hitung indikator Frequency Analyzer
            df_selected["Frequency Analyzer"] = (df_selected["Volume"] / df_selected["Frekuensi"]) ** 3

            # **Sesuaikan range Frequency Analyzer dengan harga saham**
            min_price = df_selected["Terendah"].min()
            max_price = df_selected["Tertinggi"].max()

            min_fa = df_selected["Frequency Analyzer"].min()
            max_fa = df_selected["Frequency Analyzer"].max()

            # Normalisasi Frequency Analyzer agar berada dalam rentang harga saham
            df_selected["Frequency Analyzer Scaled"] = min_price + (
                (df_selected["Frequency Analyzer"] - min_fa) / (max_fa - min_fa) * (max_price - min_price)
            )

            # **Visualisasi Candlestick**
            fig_candlestick = go.Figure()

            fig_candlestick.add_trace(go.Candlestick(
                x=df_selected["Tanggal"],
                open=df_selected["Open Price"],
                high=df_selected["Tertinggi"],
                low=df_selected["Terendah"],
                close=df_selected["Penutupan"],
                name="Candlestick"
            ))

            fig_candlestick.update_layout(
                title=f"Candlestick Chart: {selected_stock}",
                xaxis_title="Tanggal",
                yaxis_title="Harga",
                template="plotly_dark",
                xaxis_rangeslider_visible=False  # Hapus rangeslider agar lebih bersih
            )

            # **Visualisasi Frequency Analyzer dengan Overlay Harga**
            fig_analyzer = go.Figure()

            # Garis Frequency Analyzer (dalam skala harga saham)
            fig_analyzer.add_trace(go.Scatter(
                x=df_selected["Tanggal"],
                y=df_selected["Frequency Analyzer Scaled"],
                mode="lines",
                name="Frequency Analyzer",
                line=dict(color="green", width=1)
            ))

            # Overlay Harga (Harga Penutupan)
            fig_analyzer.add_trace(go.Scatter(
                x=df_selected["Tanggal"],
                y=df_selected["Penutupan"],
                mode="lines",
                name="Harga Penutupan",
                line=dict(color="orange", width=1,)  # Garis putus-putus untuk harga
            ))

            fig_analyzer.update_layout(
                title=f"Frequency Analyzer with Price Overlay",
                xaxis_title="Tanggal",
                yaxis_title="Harga",
                template="plotly_dark"
            )

            # **Tampilkan Chart**
            fig_candlestick.show()
            fig_analyzer.show()

📂 Ditemukan 68 file saham


C:\Users\User\AppData\Local\Temp\ipykernel_3768\2165600548.py:55: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [18]:
import requests
import csv
from datetime import datetime, timedelta

# API Endpoint
url = "https://pasardana.id/api/StockSearchResult/GetAll"

# Rentang tanggal yang diinginkan
start_date = datetime(2024, 11, 1)  
end_date = datetime(2025, 3, 12)    

# Header untuk request API
headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
    "Host": "pasardana.id",
    "Pragma": "no-cache",
    "Referer": "https://pasardana.id/stock/search",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, seperti Gecko) Chrome/109.0.0.0 Safari/537.36",
}

# Simpan semua data saham
all_data = []

# Looping untuk mengambil data per hari
for i in range((end_date - start_date).days + 1):
    tanggal_dicari = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
    print(f"📅 Mengambil data untuk tanggal {tanggal_dicari}...")

    # Parameter request API
    params = {
        "search": "",  # Kosong agar mengambil semua saham
        "page": 1,
        "per_page": 1000,  # Ambil sebanyak mungkin saham dalam satu request
        "sort_by": "Code",
        "sort_direction": "asc",
        "date": tanggal_dicari,
    }

    # Kirim request ke API
    response = requests.get(url, params=params, headers=headers)

    # Cek apakah request berhasil
    if response.status_code == 200:
        data = response.json()  # Data berupa list saham

        # Simpan hanya "Code", "Date", dan "AdjustedOpenPrice"
        filtered_data = [
            {
                "Code": stock["Code"],
                "Date": stock["LastDate"][:10],  # Format YYYY-MM-DD
                "AdjustedOpenPrice": stock.get("AdjustedOpenPrice", 0),
            }
            for stock in data
            if stock["LastDate"][:10] == tanggal_dicari  # Pastikan tanggal sesuai
        ]

        # Tambahkan ke daftar semua data
        all_data.extend(filtered_data)
    else:
        print(f"❌ Error {response.status_code} saat mengambil data tanggal {tanggal_dicari}, Pesan: {response.text}")

# Nama file CSV untuk menyimpan data
csv_filename = "data_saham_adjusted_open.csv"

# Simpan ke file CSV
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["Code", "Date", "AdjustedOpenPrice"])
    writer.writeheader()  # Tulis header CSV
    writer.writerows(all_data)  # Tulis data saham

print(f"\n✅ Data berhasil disimpan ke {csv_filename} dengan {len(all_data)} baris data!")

📅 Mengambil data untuk tanggal 2024-11-01...
📅 Mengambil data untuk tanggal 2024-11-02...
📅 Mengambil data untuk tanggal 2024-11-03...
📅 Mengambil data untuk tanggal 2024-11-04...
📅 Mengambil data untuk tanggal 2024-11-05...
📅 Mengambil data untuk tanggal 2024-11-06...
📅 Mengambil data untuk tanggal 2024-11-07...
📅 Mengambil data untuk tanggal 2024-11-08...
📅 Mengambil data untuk tanggal 2024-11-09...
📅 Mengambil data untuk tanggal 2024-11-10...
📅 Mengambil data untuk tanggal 2024-11-11...
📅 Mengambil data untuk tanggal 2024-11-12...
📅 Mengambil data untuk tanggal 2024-11-13...
📅 Mengambil data untuk tanggal 2024-11-14...
📅 Mengambil data untuk tanggal 2024-11-15...
📅 Mengambil data untuk tanggal 2024-11-16...
📅 Mengambil data untuk tanggal 2024-11-17...
📅 Mengambil data untuk tanggal 2024-11-18...
📅 Mengambil data untuk tanggal 2024-11-19...
📅 Mengambil data untuk tanggal 2024-11-20...
📅 Mengambil data untuk tanggal 2024-11-21...
📅 Mengambil data untuk tanggal 2024-11-22...
📅 Mengambi

In [4]:
import pandas as pd

# Baca data dari CSV
df = pd.read_csv("data_saham_ACES.csv")

# Hitung Frequency Analyzer
df["FrequencyAnalyzer"] = df["Volume"] / (df["Frequency"] ** 3)

# Urutkan berdasarkan tanggal
df = df.sort_values(by="Date")

# Buat Pine Script Header
pine_script = """\
//@version=5
indicator("Frequency Analyzer vs Close", overlay=false)

fa_data = array.new_float()
"""

# Tambahkan array analyzer ke dalam Pine Script
for val in df["FrequencyAnalyzer"]:
    pine_script += f"array.push(fa_data, {val:.4f})\n"

# Tambahkan logika plotting
pine_script += """
index = bar_index - (array.size(fa_data) - 1)
float fa_value = na
if index >= 0 and index < array.size(fa_data)
    fa_value := array.get(fa_data, index)

// Plot Frequency Analyzer (sumbu kiri)
plot(fa_value, title="Frequency Analyzer", color=color.black, linewidth=2)

// Plot harga close dari TradingView (sumbu kanan)
plot(close, title="Close Price", color=color.green, linewidth=2, axis=axis.right)
"""

# Simpan ke file .pine
with open("frequency_analyzer.pine", "w") as f:
    f.write(pine_script)

print("✅ Pine Script berhasil dibuat dan disimpan sebagai frequency_analyzer.pine")


✅ Pine Script berhasil dibuat dan disimpan sebagai frequency_analyzer.pine


In [4]:
import requests
import certifi

url = "https://api.binance.com/api/v3/klines"
params = {
    "symbol": "BTCUSDT",
    "interval": "1d",
    "limit": 5
}

try:
    response = requests.get(url, params=params, timeout=10, verify=False)
    response.raise_for_status()
    print(response.json())
except requests.exceptions.RequestException as e:
    print("Error:", e)


[[1747008000000, '104118.00000000', '105819.45000000', '100718.37000000', '102791.32000000', '31272.77792000', 1747094399999, '3242348937.38306910', 5298407, '16455.99797000', '1707457796.58813950', '0'], [1747094400000, '102791.32000000', '104976.25000000', '101429.70000000', '104103.72000000', '21253.42409000', 1747180799999, '2197244118.15236690', 3703531, '10705.97845000', '1106609664.70779500', '0'], [1747180800000, '104103.72000000', '104356.95000000', '102602.05000000', '103507.82000000', '16452.90810000', 1747267199999, '1704708279.38841090', 3025221, '8026.03333000', '831783471.65436100', '0'], [1747267200000, '103507.83000000', '104192.70000000', '101383.07000000', '103763.71000000', '17998.98604000', 1747353599999, '1850183474.52920360', 4139477, '8475.18036000', '871244345.56413600', '0'], [1747353600000, '103763.71000000', '104550.33000000', '103100.49000000', '103542.44000000', '15352.71199000', 1747439999999, '1594550287.39028940', 3143029, '7402.97409000', '769096334.75

c:\Users\User\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.binance.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
